In [1]:
import importlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import sklearn.manifold as sklm
import os
from sklearn.decomposition import PCA
import scanpy as sc
import functions
importlib.reload(functions)
functions.set_user('Kaja')

In [2]:
data = functions.call_data_clean()
ATAC = data['ATAC_seq']
RNA = data['RNA_seq']
QC = data['QC_metrics']
ATAC_scores = data['norm_scores']
exons = data['exons']
ATACtest = data['test1']
ATAC_dis = pd.read_csv("D:\\Uni\\4. Semester\\Bioinfo\\datasets\\ATAC_dis_annotated.csv", header=0, index_col=0)

In [3]:
#print(ATAC_dis.head())
#print(ATAC_dis['region_type'])
print(ATAC['genes.within.100Kb'])

ImmGenATAC1219.peakID
ImmGenATAC1219.peak_1           NaN
ImmGenATAC1219.peak_2           NaN
ImmGenATAC1219.peak_3           NaN
ImmGenATAC1219.peak_4           NaN
ImmGenATAC1219.peak_5           NaN
                              ...  
ImmGenATAC1219.peak_512591    Erdr1
ImmGenATAC1219.peak_512592    Erdr1
ImmGenATAC1219.peak_512593    Erdr1
ImmGenATAC1219.peak_512594    Erdr1
ImmGenATAC1219.peak_512595    Erdr1
Name: genes.within.100Kb, Length: 512590, dtype: object


In [ ]:
print(exons.head())

In [4]:
ATAC_100kb = ATAC_dis[ATAC_dis['genes.within.100Kb'].notna()]
ATAC_T = data['ATAC_seq_T']
ATAC_top = data['top2500std']

In [51]:
rna_df=RNA.copy()
promoters=ATAC_dis[ATAC_dis['region_type']=='Promoter']
#print(promoters)
print(promoters.index)

Index(['ImmGenATAC1219.peak_21', 'ImmGenATAC1219.peak_22',
       'ImmGenATAC1219.peak_85', 'ImmGenATAC1219.peak_86',
       'ImmGenATAC1219.peak_140', 'ImmGenATAC1219.peak_141',
       'ImmGenATAC1219.peak_145', 'ImmGenATAC1219.peak_146',
       'ImmGenATAC1219.peak_157', 'ImmGenATAC1219.peak_158',
       ...
       'ImmGenATAC1219.peak_512468', 'ImmGenATAC1219.peak_512478',
       'ImmGenATAC1219.peak_512500', 'ImmGenATAC1219.peak_512501',
       'ImmGenATAC1219.peak_512506', 'ImmGenATAC1219.peak_512507',
       'ImmGenATAC1219.peak_512508', 'ImmGenATAC1219.peak_512522',
       'ImmGenATAC1219.peak_512523', 'ImmGenATAC1219.peak_512533'],
      dtype='object', name='ImmGenATAC1219.peakID', length=28047)


In [16]:
print(rna_df.index)
print(rna_df.head())

Index(['0610007P14Rik', '0610009B22Rik', '0610009O20Rik', '0610010F05Rik',
       '0610010K14Rik', '0610012G03Rik', '0610030E20Rik', '0610037L13Rik',
       '1110004E09Rik', '1110004F10Rik',
       ...
       'Zswim4', 'Zswim8', 'Zufsp', 'Zw10', 'Zwint', 'Zyg11b', 'Zyx', 'Zzef1',
       'Zzz3', 'l7Rn6'],
      dtype='object', length=6451)
               LTHSC.34-.BM  LTHSC.34+.BM  STHSC.150-.BM  MPP4.135+.BM  \
0610007P14Rik      7.693863      7.948982       7.595628      7.681578   
0610009B22Rik      6.308741      6.318986       6.126080      6.274600   
0610009O20Rik      7.406382      7.312212       7.294084      7.548910   
0610010F05Rik      6.265552      7.436093       6.908770      7.455771   
0610010K14Rik      7.234254      7.663815       7.285670      7.338289   

               proB.CLP.BM  proB.FrA.BM  proB.FrBC.BM  B.FrE.BM    B1b.PC  \
0610007P14Rik     7.575609     7.722433      7.501594  7.116890  7.133776   
0610009B22Rik     6.742621     6.294673      6.112973  5.863

In [57]:
#print(promoters['genes.within.100Kb'])
#print(promoters.head())
promoters=promoters.T
print(promoters.index)
print(promoters.head(10))


Index(['ImmGenATAC1219.peak_21', 'ImmGenATAC1219.peak_22',
       'ImmGenATAC1219.peak_85', 'ImmGenATAC1219.peak_86',
       'ImmGenATAC1219.peak_140', 'ImmGenATAC1219.peak_141',
       'ImmGenATAC1219.peak_145', 'ImmGenATAC1219.peak_146',
       'ImmGenATAC1219.peak_157', 'ImmGenATAC1219.peak_158',
       ...
       'ImmGenATAC1219.peak_512468', 'ImmGenATAC1219.peak_512478',
       'ImmGenATAC1219.peak_512500', 'ImmGenATAC1219.peak_512501',
       'ImmGenATAC1219.peak_512506', 'ImmGenATAC1219.peak_512507',
       'ImmGenATAC1219.peak_512508', 'ImmGenATAC1219.peak_512522',
       'ImmGenATAC1219.peak_512523', 'ImmGenATAC1219.peak_512533'],
      dtype='object', name='ImmGenATAC1219.peakID', length=28047)
                        chrom   Summit mm10.60way.phastCons_scores  \
ImmGenATAC1219.peakID                                                
ImmGenATAC1219.peak_21   chr1  3671437                        0.72   
ImmGenATAC1219.peak_22   chr1  3671783                        0.91   
ImmGen

In [58]:
rows = []

for idx, row in promoters.iterrows():
    gene_list = str(row['genes.within.100Kb']).split(',')  # aufteilen
    for gene in gene_list:
        gene = gene.strip()
        new_row = row.copy()
        new_row['gene'] = gene
        rows.append(new_row)

promoters_genes_df = pd.DataFrame(rows)
promoters_genes_df = promoters_genes_df.drop(columns=['genes.within.100Kb'])  # alte Spalte raus


In [48]:
from scipy.stats import pearsonr
import pandas as pd

results = []

# Hole alle ATAC-Peak-Spalten (außer der Gene-Spalte)
peak_columns = [col for col in promoters.columns if col != 'genes.within.100Kb']

# Iteriere über alle ATAC-Peaks
for peak in peak_columns:
    # Hole ATAC-Werte für alle Zellen
    atac_values = promoters[peak]

    # Hole die Gene, die mit dem Peak assoziiert sind
    gene_list = promoters['genes.within.100Kb'].dropna().apply(lambda x: x.split(','))

    # Wir gehen Zelle für Zelle durch
    for gene_set in gene_list.unique():
        for gene in gene_set:
            gene = gene.strip()
            if gene not in rna_df.columns:
                continue

            # Hole RNA-Werte für dieses Gen
            rna_values = rna_df[gene]

            # Gemeinsame Zellen für beide Werte
            common_cells = atac_values.index.intersection(rna_values.index)

            # Hole ATAC & RNA Werte
            atac_vals = atac_values.loc[common_cells]
            rna_vals = rna_values.loc[common_cells]

            # Prüfe auf gültige Länge
            if len(atac_vals.dropna()) >= 3 and len(rna_vals.dropna()) >= 3:
                r, p = pearsonr(atac_vals, rna_vals)
                results.append({
                    'peak': peak,
                    'gene': gene,
                    'correlation': r,
                    'p_value': p,
                    'n_cells': len(common_cells)
                })

# Ergebnis als DataFrame
cor_df = pd.DataFrame(results)
cor_df = cor_df.sort_values(by='correlation', ascending=False)

# Beispiel-Ausgabe
print(cor_df.head())

KeyError: 'genes.within.100Kb'

In [43]:
from scipy.stats import pearsonr
results=[]
gene=row['genes.within.100Kb']

genes_in_promoters = set(promoters['genes.within.100Kb'])

# Gene aus rna_df (Index)
genes_in_rna = set(rna_df.index)

# Gemeinsame Gene (Schnittmenge)
common_genes = genes_in_promoters.intersection(genes_in_rna)


print(f"Anzahl Gene in beiden Datensätzen: {len(common_genes)}")
print("Beispiele gemeinsamer Gene:", list(common_genes)[:10])


for peak_id, row in promoters.iterrows():
    #peak_id=row['ImmGenATAC1219.peakID']
    gene=row['genes.within.100Kb']
    if peak_id in promoters.index and gene in rna_df.index:
        atac_values = promoters.loc[peak_id]
        rna_values= rna_df.loc[gene]

        r, p=pearsonr(atac_values, rna_values)

        results.append({
            'peak_id': peak_id,
            'gene_name': gene,
            'correlation': r,
            'p_value': p
        })

Anzahl Gene in beiden Datensätzen: 435
Beispiele gemeinsamer Gene: ['Hspa13', 'D1Ertd622e', 'Nipbl', 'Med30', 'Ufm1', 'Pkn2', 'Rgs3', 'Ccnl1', 'Raly', 'Impad1']


ValueError: `x` and `y` must have the same length along `axis`.

In [40]:
corr_df=pd.DataFrame(results)
print(corr_df.head())

Empty DataFrame
Columns: []
Index: []


In [5]:
print("ATAC_scores columns:")
print(list(ATAC_scores.columns))
print("RNA columns:")
print(list(RNA.columns))

gemeinsame_spalten = set(ATAC_scores.columns) & set(RNA.columns)
print("Gemeinsame Spalten:", gemeinsame_spalten)
print("ATAC ohne RNA:", set(ATAC_scores.columns) - set(RNA.columns))
print("RNA ohne ATAC:", set(RNA.columns) - set(ATAC_scores.columns))

ATAC_scores columns:
['LTHSC.34-.BM', 'LTHSC.34+.BM', 'STHSC.150-.BM', 'MPP4.135+.BM', 'proB.CLP.BM', 'proB.FrA.BM', 'proB.FrBC.BM', 'preB.FrD.BM', 'B.FrE.BM', 'B1b.PC', 'B.T1.Sp', 'B.T2.Sp', 'B.T3.Sp', 'B.Sp', 'B.Fem.Sp', 'B.MZ.Sp', 'B.Fo.Sp', 'B.mem.Sp', 'B.GC.CB.Sp', 'B.GC.CC.Sp', 'B.PB.Sp', 'B.PC.Sp', 'B.PC.BM', 'preT.DN1.Th', 'preT.DN2a.Th', 'preT.DN2b.Th', 'preT.DN3.Th', 'T.DN4.Th', 'T.ISP.Th', 'T.DP.Th', 'T.4.Th', 'T.8.Th', 'T.4.Nve.Sp', 'T.4.Nve.Fem.Sp', 'T.8.Nve.Sp', 'T.4.Sp.aCD3+CD40.18hr', 'Treg.4.FP3+.Nrplo.Co', 'Treg.4.25hi.Sp', 'T8.TN.P14.Sp', 'T8.IEL.LCMV.d7.SI', 'T8.TE.LCMV.d7.Sp', 'T8.MP.LCMV.d7.Sp', 'T8.Tcm.LCMV.d180.Sp', 'T8.Tem.LCMV.d180.Sp', 'NKT.Sp', 'NKT.Sp.LPS.3hr', 'NKT.Sp.LPS.18hr', 'NKT.Sp.LPS.3d', 'Tgd.g2+d17.24a+.Th', 'Tgd.g2+d17.LN', 'Tgd.g2+d1.24a+.Th', 'Tgd.g2+d1.LN', 'Tgd.g1.1+d1.24a+.Th', 'Tgd.g1.1+d1.LN', 'Tgd.Sp', 'NK.27+11b-.BM', 'NK.27+11b+.BM', 'NK.27-11b+.BM', 'NK.27+11b-.Sp', 'NK.27+11b+.Sp', 'NK.27-11b+.Sp', 'ILC2.SI', 'ILC3.NKp46-CCR6-.SI', 'I

In [41]:
results = []

# 1. Extrahiere alle Gene aus promoters (kann mehrere pro Peak sein, z.B. 'GeneA;GeneB')
genes_in_promoters_raw = promoters['genes.within.100Kb'].dropna().tolist()

genes_in_promoters = set()
for gstr in genes_in_promoters_raw:
    genes_in_promoters.update(gstr.split(';'))

genes_in_rna = set(rna_df.index)
common_genes = genes_in_promoters.intersection(genes_in_rna)

print(f"Anzahl Gene in beiden Datensätzen: {len(common_genes)}")
print("Beispiele gemeinsamer Gene:", list(common_genes)[:10])

# 2. Für jeden Promoter-Peak Korrelation berechnen
for peak_id, row in promoters.iterrows():
    gene_str = row['genes.within.100Kb']
    if pd.isna(gene_str):
        continue
    
    genes_list = gene_str.split(';')
    
    # Suche erstes Gen aus der Schnittmenge common_genes
    gene = None
    for g in genes_list:
        if g in common_genes:
            gene = g
            break
    if gene is None:
        continue
    
    # Promoter ATAC-Signalwerte (alles außer Annotationen)
    # Angenommen, deine Annotationsspalten sind z.B. 'genes.within.100Kb' und andere,
    # und die restlichen Spalten sind Proben mit Signalwerten.
    # Deshalb hier alle Spalten außer Annotationen filtern:
    atac_signal = row.drop(labels=['genes.within.100Kb', 'andere_annot_spalten...'], errors='ignore')
    
    rna_values = rna_df.loc[gene]
    
    if atac_signal.isnull().any() or rna_values.isnull().any():
        continue
    
    r, p = pearsonr(atac_signal, rna_values)
    
    results.append({
        'peak_id': peak_id,
        'gene_name': gene,
        'correlation': r,
        'p_value': p
    })

corr_df = pd.DataFrame(results)
print(f"Korrelationen berechnet für {len(corr_df)} Promoter-Gene Paare")

Anzahl Gene in beiden Datensätzen: 435
Beispiele gemeinsamer Gene: ['Hspa13', 'D1Ertd622e', 'Nipbl', 'Med30', 'Ufm1', 'Pkn2', 'Rgs3', 'Ccnl1', 'Raly', 'Impad1']


NameError: name 'pearsonr' is not defined

In [6]:
def correlate_promoter_atac_rna(ATAC, RNA, ATAC_scores):
    # Nur gemeinsame Cell-Typ-Namen verwenden!
    gemeinsame_celltypen = [col for col in ATAC_scores.columns if col in RNA.columns]
    print(f"Gemeinsame Celltypen ({len(gemeinsame_celltypen)}):", gemeinsame_celltypen)
    
    promoter_peaks = ATAC[ATAC['distance_to_TSS'] <= 2000].copy()
    promoter_peaks['main_gene'] = promoter_peaks['nearest_gene']
    results = []
    for idx, row in promoter_peaks.iterrows():
        gene = row['main_gene']
        if gene in RNA.index:
            atac_vec = row[gemeinsame_celltypen].values.astype(float)
            rna_vec = RNA.loc[gene, gemeinsame_celltypen].values.astype(float)
            if not (np.isnan(atac_vec).any() or np.isnan(rna_vec).any()):
                r, p = scipy.stats.pearsonr(atac_vec, rna_vec)
                results.append({'peak_id': idx, 'gene': gene, 'r': r, 'p': p})
    print(f"Anzahl berechnete Korrelationen: {len(results)}")
    return pd.DataFrame(results)

In [8]:
print(ATAC['distance_to_TSS'])

KeyError: 'distance_to_TSS'

In [7]:
cor_df = correlate_promoter_atac_rna(ATAC_dis[ATAC_dis.index.isin(ATAC_top)], RNA, ATAC_scores)
print(cor_df.columns)
print(cor_df['peak_id'])

Gemeinsame Celltypen (85): ['LTHSC.34-.BM', 'LTHSC.34+.BM', 'STHSC.150-.BM', 'MPP4.135+.BM', 'proB.CLP.BM', 'proB.FrA.BM', 'proB.FrBC.BM', 'B.FrE.BM', 'B1b.PC', 'B.T1.Sp', 'B.T2.Sp', 'B.T3.Sp', 'B.Sp', 'B.Fem.Sp', 'B.MZ.Sp', 'B.Fo.Sp', 'B.mem.Sp', 'B.GC.CB.Sp', 'B.GC.CC.Sp', 'B.PB.Sp', 'B.PC.Sp', 'B.PC.BM', 'preT.DN1.Th', 'preT.DN2a.Th', 'preT.DN2b.Th', 'preT.DN3.Th', 'T.DN4.Th', 'T.ISP.Th', 'T.DP.Th', 'T.4.Th', 'T.8.Th', 'T.4.Nve.Sp', 'T.4.Nve.Fem.Sp', 'T.8.Nve.Sp', 'T.4.Sp.aCD3+CD40.18hr', 'Treg.4.FP3+.Nrplo.Co', 'Treg.4.25hi.Sp', 'T8.TN.P14.Sp', 'T8.TE.LCMV.d7.Sp', 'T8.MP.LCMV.d7.Sp', 'T8.Tcm.LCMV.d180.Sp', 'T8.Tem.LCMV.d180.Sp', 'NKT.Sp', 'NKT.Sp.LPS.3hr', 'NKT.Sp.LPS.18hr', 'NKT.Sp.LPS.3d', 'Tgd.g2+d17.24a+.Th', 'Tgd.g2+d17.LN', 'Tgd.g2+d1.24a+.Th', 'Tgd.g2+d1.LN', 'Tgd.g1.1+d1.24a+.Th', 'Tgd.g1.1+d1.LN', 'Tgd.Sp', 'NK.27+11b-.BM', 'NK.27+11b+.BM', 'NK.27-11b+.BM', 'NK.27+11b-.Sp', 'NK.27+11b+.Sp', 'NK.27-11b+.Sp', 'ILC2.SI', 'ILC3.NKp46-CCR6-.SI', 'ILC3.NKp46+.SI', 'ILC3.CCR6+.SI

In [10]:
gene_df=exons.copy()
gene_df['TSS'] = np.where(gene_df['Strand'] == '+', gene_df['Transcription Start'], gene_df['Transcription End'])

In [3]:
import pandas as pd

# Beispiel: du hast exons.csv und atac.csv eingelesen
# exons = pd.read_csv("exons.csv")
# ATAC = pd.read_csv("atac.csv")

results = []

for i, peak in ATAC.iterrows():
    peak_chr = peak['chrom']
    peak_summit = peak['Summit']
    
    # Falls mehrere Gene in einem Feld mit Komma getrennt sind
    genes = str(peak['genes.within.100Kb']).split(',')
    
    for gene in genes:
        gene = gene.strip()
        gene_row = exons[(exons['Genname'] == gene) & (exons['Chromosom'] == peak_chr)]
        
        if gene_row.empty:
            continue
        
        strand = gene_row.iloc[0]['Strand']
        
        # TSS je nach Strang
        if strand == '+':
            tss = gene_row.iloc[0]['Transcription Start']
            distance = peak_summit - tss
        else:
            tss = gene_row.iloc[0]['Transcription End']
            distance = tss - peak_summit
        
        results.append({
            'peak_id': peak['ImmGenATAC1219.peakID'],
            'chrom': peak_chr,
            'gene': gene,
            'strand': strand,
            'summit': peak_summit,
            'tss': tss,
            'distance_to_TSS': distance,
            'abs_distance': abs(distance)
        })

# Umwandeln in DataFrame
dist_df = pd.DataFrame(results)

# Beispiel: nächstgelegene CRE pro Gen
closest_CRE_per_gene = dist_df.sort_values(by='abs_distance').groupby('gene').first().reset_index()

# Anzeigen
print(closest_CRE_per_gene.head())

KeyError: 'ImmGenATAC1219.peakID'

In [8]:
atac_df=ATAC.copy()
results=[]
for i, peak in atac_df.iterrows():
    chrom = peak['chrom']
    summit = peak['Summit']
    genes = str(peak['genes.within.100Kb']).split(',')
    for gene in genes:
        gene = gene.strip()

        match = exons[(exons['Genname'] == gene) & (exons['Chromosom'] == chrom)]
            
        if match.empty:
            continue

        row = match.iloc[0]
        strand = row['Strand']
        tss = row['Transcription Start'] if strand == '+' else row['Transcription End']
        distance = summit - tss if strand == '+' else tss - summit
        results.append({
            'peak_id': peak['peak_id'],
            'gene': gene,
            'chrom': chrom,
            'strand': strand,
            'summit': summit,
            'tss': tss,
            'distance_to_TSS': distance,
            'abs_distance': abs(distance)

        })
distance_df = pd.DataFrame(results)

KeyError: 'peak_id'